In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt
from math import log10, sqrt
import os
from skimage import exposure
from tifffile import imsave
import random
def hcf(a,b):
    if(b%a==0):
        return a
    else:
        return hcf(b%a,a)
def randomseq(size):
    random.seed(size)
    
    num=[x for x in range(size)]
    h=len(num)
    for m in range(h):
        if m>2:
            r=hcf(m,h)
            if r==1:
                break 
    i=random.randint(0,size-1)    
    x=i
    res=[]
    while True:
        i=(i+m)%h
        if i==x:
            break
        res.append(i)
        
    res.append(x)
    return res
def embedding(img,wtr_img):
    wrow,wcol=wtr_img.shape
    LL,(HL,LH,HH)=pywt.dwt2(img,wavelet='haar')
    l2x2=LL.reshape(-1,2,2)
    sstore=[]
    ustore=[]
    vstore=[]
    for i in l2x2:
        u,s,v=np.linalg.svd(i)
        ustore.append(u)
        vstore.append(v)
        sstore.append(s)
    ss=np.array(sstore)
    ss_backup=ss.copy()
    h2x2=HH.reshape(-1,2,2) #Store the ss_backup into HH
    uu=[]
    sss=ss_backup.ravel()
    maxs=max(sss)
    stemp=[]
    vv=[]
    newHH=[]
    for i in h2x2:
        u,s,v=np.linalg.svd(i)
        uu.append(u)
        vv.append(v)
        stemp.append(s)
    stemp=np.array(stemp)
    stemp=ss_backup
    for i in range(len(stemp)):
        u=uu[i]
        v=vv[i]
        s=stemp[i]/maxs
        temp=u@np.diag(s)@v
        newHH.append(temp)
    newHH=np.array(newHH)
    newHH=newHH.ravel()
    newHH=newHH.reshape(256,256)
    HH=newHH
    wtr_list=wtr_img.ravel()
    j=0
    seqnum=randomseq(wrow*wcol)
    for i in seqnum:
        ss[i][0]+=wtr_list[j]/256
        j+=1    
    newll=[]
    for i in range(len(ss)):
        u=ustore[i]
        v=vstore[i]
        s=ss[i]
        temp=u@np.diag(s)@v
        newll.append(temp)
    newll=np.array(newll)
    newll=newll.ravel()
    row,col=LL.shape
    newll=newll.reshape(row,col)
    watermarked_img=pywt.idwt2((newll,(HL,LH,HH)),wavelet='haar')
    return watermarked_img,maxs
def recover(watermarked_img,maxs,wrow,wcol):
    dLL,(dHL,dLH,dHH)=pywt.dwt2(watermarked_img,wavelet='haar')
    dl2x2=dLL.reshape(-1,2,2)
    dh2x2=dHH.reshape(-1,2,2)
    sstore2=[]
    ustore=[]
    vstore=[]
    uu2=[]
    vv2=[]
    ss2=[]
    for i in dh2x2:
        u,s,v=np.linalg.svd(i)
        # s=s.round()
        uu2.append(u)
        vv2.append(v)
        ss2.append(s*maxs)
    ss2=np.array(ss2)
    for i in dl2x2:
        u,s,v=np.linalg.svd(i)
        # s=s.round()
        ustore.append(u)
        vstore.append(v)
        sstore2.append(s)
    sstore2=np.array(sstore2)
    rec_wtr=[]
    seqnum=randomseq(wrow*wcol)
    for i in seqnum:
        temp=(sstore2[i][0]-ss2[i][0])*256
        rec_wtr.append(temp)
    rec_wtr=np.array(rec_wtr)
    final_img=rec_wtr.reshape(wrow,wcol)
    temp_im=np.round(final_img)
    for i in range(len(temp_im)):
        for j in range(len(temp_im[0])):
            temp_im[i][j]=int(temp_im[i][j])
    return temp_im
def img_read(x):
    img=cv2.imread(x,0)
    return img
def watermark_read(x):
    wtr_img=cv2.imread(x,0)
    wtr_img=cv2.resize(wtr_img,(128,128))
    return wtr_img
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity
from PIL import Image

def ssim(name,name1):
    return structural_similarity(name,name1)
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    try:
        max_pixel = 255.0
        psnr = 20 * log10(max_pixel / sqrt(mse))
        return psnr
    except:
        return -9999  #When MSE is Zero, i.e, no noise is present in the image
def norm_data(data):
    mean_data=np.mean(data)
    std_data=np.std(data,ddof=1)
    return (data-mean_data)/(std_data)
def ncc(data0,data1):
    c=(1.0/(data0.size-1))*np.sum(norm_data(data0)*norm_data(data1))
    return c


In [2]:
images=os.listdir('../datasets/BOSSbase_1.01_DATASET/BOSSbase_1.01')
watermarks=os.listdir('./CoverImages')
imgpath='../datasets/BOSSbase_1.01_DATASET/BOSSbase_1.01'
wtrpath='./CoverImages'
ssim_list=[]
psnr_list=[]
imagelist=[]
watermarklist=[]
ncc_list=[]
c=0
test_wtr=[watermarks[0]]

# i=watermarks[0]
for i in test_wtr:
    count=0
    wtr_img=watermark_read(os.path.join(wtrpath,i))
    name_folder=i.split('.')
    name_folder=name_folder[0]
    path=f'D:/CS/Sem 6/Project/Practice/Stego_Image_Generation/{name_folder}'
    os.mkdir(path)
    for j in images:
        img=img_read(os.path.join(imgpath,j))
        watermarked_img,stego_key=embedding(img,wtr_img)
        wrow,wcol=wtr_img.shape
        name="wtr_"+i+'_host_'+j
        imsave(os.path.join(path,f'{name}_stego.tif'),watermarked_img.astype('uint8'))
        generated_img=recover(watermarked_img,stego_key,wrow,wcol)
        count+=1
        if count>=1000:
            break
    c+=1


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/CS/Sem 6/Project/Practice/Stego_Image_Generation/airplane'